In [1]:
!pip install transformers
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 2.1MB 5.8MB/s 
     |████████████████████████████████| 901kB 51.1MB/s 
     |████████████████████████████████| 3.3MB 57.1MB/s 
Mounted at /content/drive


In [2]:
import json
import sys
import numpy as np
import time
import torch
import os
import torch.nn as nn
import pdb
import time
import gc
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
device = "cuda" if torch.cuda.is_available() else "cpu"
print(sys.path)
import pandas as pd
from collections import defaultdict

from transformers import (
    DistilBertTokenizer,
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    DistilBertForMaskedLM,
)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokenizer_options = {
    "return_tensors": "pt",
    "truncation": True,
    "padding": True,
    "max_length": 512,
}

model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load("/content/drive/MyDrive/585/demo_model.pt", map_location=device))
model.to(device)
model.eval()

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): L

In [4]:
# preparing for the queries dict

diverse_queries = pd.read_csv("/content/drive/MyDrive/585/diverse.queries.all.tsv", sep="\t", header=None)

query_dict = defaultdict(str)
passage_dict = defaultdict(str)

for qid, q in zip(diverse_queries[0], diverse_queries[1]):
    query_dict[qid] = q

diverse_queries.head()

,0,1
0,1006418,which cal state schools have football teams
1,291569,how many pairs of chromosomes in a human cell
2,736625,what is cultured stone
3,1043711,who make the land rover
4,16151,amara name meaning


In [5]:
# preparing for the passages dict

diverse_passages = pd.read_csv("/content/drive/MyDrive/585/diverse.passages.all.tsv", sep="\t", header=None)

query_dict = defaultdict(str)
passage_dict = defaultdict(str)

# preparing passage dict and query dict from top 1000 dev
for pid, p in zip(diverse_passages[0], diverse_passages[1]):
    passage_dict[pid] = p
    
diverse_passages.head()

,0,1
0,140,"Charlottesville, VA: CBSA Type: Metro: CBSA Di..."
1,222,"For example, the pressure of electromagnetic r..."
2,223,"The evolution of a gas, a color change, the fo..."
3,228,"In physics, a photon is usually denoted by the..."
4,264,Lakewood Village Population by Race. The table...


In [6]:
# preparing for the relevant and test set

test_triplets = pd.read_csv("/content/drive/MyDrive/585/diverse.triplets.test.tsv", sep="\t", header=None)
test_triplets

small_dict_rel = defaultdict(list)

for qid, pid in zip(test_triplets[0], test_triplets[1]):
    if pid not in list(small_dict_rel[qid]):
      small_dict_rel[qid].append(pid)

small_dict_test = defaultdict(list)

for qid, pos,neg in zip(test_triplets[0], test_triplets[1],test_triplets[2]):
    if pos not in list(small_dict_test[qid]):
      small_dict_test[qid].append(pos)
    if neg not in list(small_dict_test[qid]):
      small_dict_test[qid].append(neg)

In [7]:
preds_dict = defaultdict(dict)

with torch.no_grad():

    for idx, (qid, pids) in enumerate(small_dict_test.items()):
      docs = [passage_dict[pid] for pid in pids]
      queries = [query_dict[qid]]

      inputs = list(queries) + list(docs) 
      encodings = tokenizer(inputs, **tokenizer_options).to(device)
      ids, masks = encodings["input_ids"], encodings["attention_mask"]

      ids = ids.to(device)  # (Q + D, MAXLEN) 
      masks = masks.to(device)

      outputs = model.distilbert(ids, masks)
      outputs_hidden = outputs.last_hidden_state.mean(dim=1) #[:, 0]

      vec_queries = outputs_hidden[:len(queries)].unsqueeze(1)
      vec_documents = outputs_hidden[len(queries):].unsqueeze(0)

      # Compute pairwise distances between queries and documents
      all_distances, all_indices = (vec_queries - vec_documents).norm(dim=-1).sort(dim=-1)
      
      # Print nicely the results
      for qdistances, dindices, query in zip(all_distances, all_indices, queries):
          rank_dict = preds_dict[qid]
          for dist, dindex in zip(qdistances, dindices):
              rank_dict[dist.item()] = small_dict_test[qid][dindex]
          preds_dict[qid] = rank_dict

      del outputs, ids, masks, outputs_hidden, encodings, all_distances, all_indices
      

In [13]:
def compute_metrics(qids_to_relevant_passageids, qids_to_ranked_candidate_passages, MaxMRRRank):
    """Compute MRR metric
    Args:    
    p_qids_to_relevant_passageids (dict): dictionary of query-passage mapping
        Dict as read in with load_reference or load_reference_from_stream
    p_qids_to_ranked_candidate_passages (dict): dictionary of query-passage candidates
    Returns:
        dict: dictionary of metrics {'MRR': <MRR Score>}
    """
    all_scores = {}
    MRR = 0
    qids_with_relevant_passages = 0
    ranking = []
    for qid in qids_to_ranked_candidate_passages:
        if qid in qids_to_relevant_passageids:
            ranking.append(0)
            target_pid = qids_to_relevant_passageids[qid]
            candidate_pid = qids_to_ranked_candidate_passages[qid]

            if len(candidate_pid) > 0 :
              if len(candidate_pid) < MaxMRRRank:
                MaxMRRRank = len(candidate_pid)

              for i in range(0,MaxMRRRank):
                if candidate_pid[i] in target_pid:
                    MRR += 1/(i + 1)
                    ranking.pop()
                    ranking.append(i+1)
                    break
    print(MRR)
    if len(ranking) == 0:
        raise IOError("No matching QIDs found. Are you sure you are scoring the evaluation set?")
    
    MRR = MRR/len(qids_to_relevant_passageids)
    all_scores['MRR @20'] = MRR
    all_scores['QueriesRanked'] = len(qids_to_ranked_candidate_passages)
    return all_scores

In [9]:
# preparing the final dict: qid,pid by sorting all ranking for each qid
pred_dict_final = defaultdict(list)

for key in preds_dict.keys():
  rank_dict = preds_dict[key]
  
  for pred_key in rank_dict.keys():
    pred_dict_final[key].append(rank_dict[pred_key])

In [14]:
MaxMRRRank = 20
compute_metrics(small_dict_rel, pred_dict_final, MaxMRRRank)

89.99732924883872


{'MRR @20': 0.1785661294619816, 'QueriesRanked': 504}

In [12]:
MaxMRRRank = 10
compute_metrics(small_dict_rel, pred_dict_final, MaxMRRRank)

74.3436507936509


{'MRR @10': 0.14750724363819623, 'QueriesRanked': 504}